In [ ]:
%load_ext autoreload
%autoreload 2

import os,sys
sys.path.append(os.path.abspath(".."))
sys.path.append(os.path.abspath("../.."))
sys.path.append(os.path.abspath("../training"))
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import os
os.environ["EXP_NAME"] = "movie_recom"
os.environ["MAX_RATING"] = "5"
os.environ["MLFLOW_TRACKING_URI"] = "http://localhost:8081"
os.environ["EPOCH"] = "2"

os.environ["AWS_DEFAULT_REGION"] = "us-east-1"
os.environ["AWS_ACCESS_KEY_ID"] = "minio"
os.environ["AWS_SECRET_ACCESS_KEY"] = "minio123"
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "http://localhost:9000"

In [ ]:
from training.xgb import main, test_xgb_model
from movie_recommender.workflow import read_parquet_from_s3, connect_minio,connect_mlflow

DB_MINIO_BUCKET = "trainingbucket"
connect_minio()
connect_mlflow()

In [ ]:
ratings = read_parquet_from_s3(DB_MINIO_BUCKET, "ratings.parquet")
movies = read_parquet_from_s3(DB_MINIO_BUCKET, "movies.parquet")
ratings = ratings.merge(movies, on="movie_id")
ratings.rating *= 5

In [ ]:
booster = main(ratings,n_trials=1)

In [ ]:
test_xgb_model()